In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

import keras
import keras.backend as K
import tensorflow as tf
from keras import applications
from keras.models import Model
from keras.layers import Flatten, Dense, Input,concatenate
from keras.optimizers import Adam
from keras.models import load_model, model_from_json

import random
#from PIL import Image, ImageChops

In [66]:
#gen="./input/handwritten-signatures/sample_Signature/genuine"
#forg="./input/handwritten-signatures/sample_Signature/forged"
gen="../input/handwritten-signatures/sample_Signature/genuine"
forg="../input/handwritten-signatures/sample_Signature/forged"

#gentr="../input/sigcomp-2009-train/sigcomp 2009 train/Sigcomp 2009 train/genuine"
#forgtr="../input/sigcomp-2009-train/sigcomp 2009 train/Sigcomp 2009 train/forgeries"

#gent="../input/sigcomp-2009/sigcomp 2009/genuines"
#forgt="../input/sigcomp-2009/sigcomp 2009/forgeries"
#training set
gentr="../input/sigcomp-2009-train/genuine"
forgtr="../input/sigcomp-2009-train/forgeries"

#test set
gent="../input/sigcomp-2009/genuines"
forgt="../input/sigcomp-2009/forgeries"

In [57]:
img_width, img_height, channels = 224, 224, 3

dim = (img_width, img_height)

def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

def returnimages(path,img):
    image=cv2.imread(path+"/"+ img)                  #bringing the image
    image=cv2.resize(image, (img_width, img_height))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=to_rgb(image).reshape(1,img_width, img_height,3)/255.0       #resizing and normalizing    
    return image     

def getfiles(num,gen,forg):
    a=os.listdir(gen)
    b=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    
    n,m=[],[]
    for i in b:
        if i.endswith(c+".png"):
            n=n+[i]
        elif i.endswith(c+".PNG"):
            n=n+[i]
    for i in a:
        if i.endswith(c+".png"):
            m=m+[i]
        elif i.endswith(c+".PNG"):
            m=m+[i]
    return m.pop(),n,m

def getfiles2(num):
    a=os.listdir(gentr)
    b=os.listdir(forgtr)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    n,m=[],[]
    for i in b:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            n=n+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            n=n+[i]
    for i in a:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            m=m+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            m=m+[i]
    return m.pop(),n,m



In [58]:
def triplet_loss(y_true, y_pred):
    alpha = 0.5
    anchor, positive, negative =y_pred[0,0:512], y_pred[0,512:1024], y_pred[0,1024:1536]
    
    positive_distance = K.mean(K.square(anchor - positive),axis=-1)
    negative_distance = K.mean(K.square(anchor - negative),axis=-1)
    return K.mean(K.maximum(0.0, positive_distance - negative_distance + alpha))

In [30]:
model = applications.vgg19.VGG19(weights='imagenet', include_top=False, pooling='max')

In [31]:
for layer in model.layers[:15]:
    layer.trainable = False

In [32]:
anchor_in = Input(shape=(img_width, img_height, channels))
pos_in = Input(shape=(img_width, img_height, channels))
neg_in = Input(shape=(img_width, img_height, channels))

anchor_out = model(anchor_in)
pos_out = model(pos_in)
neg_out = model(neg_in)
merged_vector = concatenate([anchor_out, pos_out, neg_out],axis=1)

In [33]:
model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)

In [34]:
model.compile(optimizer=Adam(lr=0.000005),loss=triplet_loss)

In [59]:
def generator():
    for i in range(1,31):
        if(i<10):
            anc,neg,pos=getfiles(float("0.00"+str(i)),gen,forg)
        else:
            anc,neg,pos=getfiles(float("0.0"+str(i)),gen,forg)
        for i in range(len(neg)):
            for j in range(len(pos)):
                anchor=returnimages(gen,anc)
                positive=returnimages(gen,pos[j])
                negative=returnimages(forg,neg[i])
               # yield ([anc,pos[j],neg[i]],[0])
                yield ([anchor,positive,negative],[0])

In [36]:
for x in range(2):
    model.fit_generator(generator(),steps_per_epoch=200,epochs=3)

Epoch 1/3
200/200 [==============================] - 225s 1s/step - loss: 0.1818
Epoch 2/3
200/200 [==============================] - 223s 1s/step - loss: 0.0242
Epoch 3/3
200/200 [==============================] - 227s 1s/step - loss: 0.0301
Epoch 1/3
200/200 [==============================] - 229s 1s/step - loss: 0.0032
Epoch 2/3
200/200 [==============================] - 236s 1s/step - loss: 0.0000e+00
Epoch 3/3
200/200 [==============================] - 235s 1s/step - loss: 0.0000e+00


In [60]:
model.compile(optimizer=Adam(lr=0.000002),loss=triplet_loss)

In [61]:
def generator2():
    x=["0.001","0.004", "0.005", "0.006", "0.007",
       "0.008", "0.009", "0.010", "0.011", "0.001", "0.010"]
  #  x=["0.001", "0.004", "0.006", "0.010"]

    for k in x:
        anc,neg,pos=getfiles2(k)
        frac=0.95    
        inds = set(random.sample(list(range(len(neg))), int(frac*len(neg))))
        neg = [n for i,n in enumerate(neg) if i not in inds]
    
        for i in range(len(neg)):
            for j in range(len(pos)):
                anchor=returnimages(gentr,anc)
                positive=returnimages(gentr,pos[j])
                negative=returnimages(forgtr,neg[i])
               # yield ([anc,pos[j],neg[i]])
                yield ([anchor,positive,negative],[0])

In [67]:
for x in range(2):
    model.fit_generator(generator2(),steps_per_epoch=32,epochs=11)

Epoch 1/11
32/32 [==============================] - 38s 1s/step - loss: 0.0111
Epoch 2/11
32/32 [==============================] - 36s 1s/step - loss: 0.0000e+00
Epoch 3/11
32/32 [==============================] - 36s 1s/step - loss: 0.0000e+00
Epoch 4/11
32/32 [==============================] - 36s 1s/step - loss: 0.0000e+00
Epoch 5/11
32/32 [==============================] - 36s 1s/step - loss: 0.0382
Epoch 6/11
32/32 [==============================] - 36s 1s/step - loss: 0.0724
Epoch 7/11
32/32 [==============================] - 37s 1s/step - loss: 0.0422
Epoch 8/11
32/32 [==============================] - 38s 1s/step - loss: 0.0394
Epoch 9/11
32/32 [==============================] - 41s 1s/step - loss: 0.0265
Epoch 10/11
32/32 [==============================] - 39s 1s/step - loss: 0.0016
Epoch 11/11
32/32 [==============================] - 36s 1s/step - loss: 0.0012
Epoch 1/11
32/32 [==============================] - 1008s 31s/step - loss: 0.0000e+00
Epoch 2/11
32/32 [=============

In [68]:
tneg,tpos=0,0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]
for k in x: #the id of signatures you want to check
    #print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    tneg=tneg+len(neg)
    tpos=tpos+len(pos)
print(tneg,tpos)

624 203


In [75]:
os.listdir('../working')

[]

In [76]:
# Save the weights
model.save_weights('model_weights.h5')

# Save the model architecture
#with open('model_architecture.json', 'w') as f:
#    f.write(model.to_json())

In [77]:
os.listdir('../working')

[]

In [78]:
forg_passed=0
gen_flagged=0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    
    anchor=returnimages(gent,anc)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor,positive,anchor])
        a, p, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        dist=np.linalg.norm(a-p)
        #print("positive distance is ",dist)
        if(dist>32):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor,negative,anchor])
        a, n, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist=np.linalg.norm(a-n)
        #print("negative distance is ",dist)
        if(dist>32):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)

When k is  0.002
gen flagged -  34.16919 file name is -  NFI-00207002.png
forg passed -  31.759441 file name is -  NFI-08901002.png
When k is  0.008
gen flagged -  39.072468 file name is -  NFI-00801008.png
gen flagged -  36.131927 file name is -  NFI-00803008.png
gen flagged -  33.295662 file name is -  NFI-00811008.png
gen flagged -  32.218952 file name is -  NFI-00810008.png
gen flagged -  34.052654 file name is -  NFI-00804008.png
forg passed -  31.130236 file name is -  NFI-05201008.png
When k is  0.016
gen flagged -  33.33131 file name is -  NFI-01607016.png
gen flagged -  35.829453 file name is -  NFI-01605016.png
gen flagged -  33.366035 file name is -  NFI-01601016.png
gen flagged -  34.542187 file name is -  NFI-01604016.png
gen flagged -  32.489124 file name is -  NFI-01608016.png
gen flagged -  32.335926 file name is -  NFI-01602016.png
forg passed -  30.157505 file name is -  NFI-07406016.png
forg passed -  30.676947 file name is -  NFI-08505016.png
forg passed -  29.14809

In [79]:
def getfilest(num,gen,forg):
    a=os.listdir(gen)
    b=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    
    n,m=[],[]
    for i in b:
        if i.endswith(c+".png"):
            n=n+[i]
        elif i.endswith(c+".PNG"):
            n=n+[i]
    for i in a:
        if i.endswith(c+".png"):
            m=m+[i]
        elif i.endswith(c+".PNG"):
            m=m+[i]
    return m.pop(),m.pop(),n,m

In [80]:
forg_passed=0
gen_flagged=0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc1,anc2,neg,pos=getfilest(k,gent,forgt)
    
    anchor1=returnimages(gent,anc1)
    anchor2=returnimages(gent,anc2)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor1,positive,anchor2])
        a1, p, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        dist1=np.linalg.norm(a1-p)
        dist2=np.linalg.norm(a2-p)
        dist=(dist1+dist2)/2
        #print("positive distance is ",dist)
        if(dist>32):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor1,negative,anchor2])
        a1, n, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist1=np.linalg.norm(a1-n)
        dist2=np.linalg.norm(a2-n)
        #print("negative distance is ",dist)
        dist=(dist1+dist2)/2
        if(dist>32):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)

When k is  0.002
gen flagged -  34.20097351074219 file name is -  NFI-00207002.png
forg passed -  30.962337493896484 file name is -  NFI-08901002.png
When k is  0.008
gen flagged -  35.70679473876953 file name is -  NFI-00801008.png
gen flagged -  32.267417907714844 file name is -  NFI-00803008.png
When k is  0.016
gen flagged -  33.771759033203125 file name is -  NFI-01608016.png
gen flagged -  34.40605926513672 file name is -  NFI-01612016.png
forg passed -  31.67192840576172 file name is -  NFI-07704016.png
forg passed -  30.936824798583984 file name is -  NFI-07706016.png
When k is  0.018
gen flagged -  37.429237365722656 file name is -  NFI-01807018.png
gen flagged -  37.59374237060547 file name is -  NFI-01811018.png
gen flagged -  34.5028190612793 file name is -  NFI-01801018.png
gen flagged -  40.34123992919922 file name is -  NFI-01804018.png
When k is  0.024
When k is  0.033
gen flagged -  32.24171447753906 file name is -  NFI-03312033.png
forg passed -  29.91632843017578 fil